In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import os
import yaml
import itertools
import shutil
from datetime import datetime
from pathlib import Path
from openpyxl.styles import PatternFill
from openpyxl import load_workbook
from collections import defaultdict
import sys
from openpyxl.styles import Border, Side
from openpyxl.styles import Alignment, Font
from openpyxl.utils import get_column_letter

In [ ]:
# cores list of sites for AA

project = 'Greenland'
output_dir = '/Users/quinnmackay/Desktop/table_out'

# get all link combos
with open(f'/Users/quinnmackay/Documents/GitHub/BICC/Antarctic Chronology Accuracy Project/{project}/parameters.yml') as f:
    data = yaml.safe_load(f)
list_sites = data["list_sites"]
pairs = [f"{a}-{b}" for a, b in itertools.combinations(list_sites, 2)]

error_margin = 0.1
big_error_margin = 0.25
base_core_age = 'EDC'

In [3]:
big_table = pd.DataFrame()

for core in list_sites: # loop through each core
    for comparison_core in list_sites: # loop through each core other than the initial load
        pair = f"{core}-{comparison_core}"
        if core != comparison_core and pair in pairs: # make sure not the same core and we skip non-existent linkages
            pair_dir = Path(f'/Users/quinnmackay/Documents/GitHub/BICC/Antarctic Chronology Accuracy Project/{project}/{pair}')

            # Check: directory exists AND contains at least one .txt file
            txt_files = list(pair_dir.glob("*.txt"))
            if not pair_dir.is_dir() or not txt_files:
                continue

            dfs=[] #load all text files into one
            for txt in txt_files:
                df = pd.read_csv(txt, sep="\t", comment="#")
                dfs.append(df)
    
            num_files = len(dfs)
            load_data = pd.concat(dfs, ignore_index=True)
            original_rows = len(load_data)

            drop_rows = []
            drop_rows_merge = set()
            new_merged_rows = []
            for idx, row in load_data.iterrows():

                mask1 = abs(row['depth1'] - load_data['depth1']) <= error_margin
                mask1[idx] = False
                mask2 = abs(row['depth2'] - load_data['depth2']) <= error_margin 
                mask2[idx] = False

                close_points = load_data[mask1 & mask2]
                num_close = len(close_points)
                close_idxs = load_data.index[mask1 & mask2]

                if num_close > 0:
                    refs = [load_data.at[idx, 'reference']] + [load_data.at[i, 'reference'] for i in close_idxs] #adjoin references
                    merged_ref = "; ".join(str(r) for r in refs if pd.notna(r))

                    depth1_vals = [load_data.at[idx, 'depth1']] + [load_data.at[i, 'depth1'] for i in close_idxs]
                    merged_depth1 = np.mean(depth1_vals)

                    depth2_vals = [load_data.at[idx, 'depth2']] + [load_data.at[i, 'depth2'] for i in close_idxs]
                    merged_depth2 = np.mean(depth2_vals)

                    new_merged_rows.append({'reference': merged_ref, 'depth1': merged_depth1, 'depth2': merged_depth2}) #create new merged row

                    drop_rows_merge.add(idx)
                    for i in close_idxs:
                        drop_rows.append(i)
                        if drop_rows.count(i) >= num_files:
                            print(f'WARNING: Row {load_data.at[i, 'depth1']} | {load_data.at[i, 'depth2']} for {pair}. Reference {load_data.at[i, 'reference']}.')
                            print(f'Called by row {load_data.at[idx, 'depth1']} | {load_data.at[idx, 'depth2']} from reference {load_data.at[idx, 'reference']}.')

            # drop duplicate rows
            drop_rows = set(drop_rows).union(drop_rows_merge)
            load_data = load_data.drop(index=drop_rows).reset_index(drop=True)
            # add merged rows
            merged_df = pd.DataFrame(new_merged_rows)
            load_data = pd.concat([load_data, merged_df], ignore_index=True)
            load_data.drop_duplicates(subset=['depth1', 'depth2'], inplace=True)
            load_data = load_data.reset_index(drop=True)

            load_data = load_data.sort_values(by=['depth1']).reset_index(drop=True)

            # rename to create unique columns for this pair
            load_data = load_data.rename(columns={
                'depth1': f"{pair}_{core}",
                'depth2': f"{pair}_{comparison_core}",
                'reference': f"{pair}_reference"
            })

            print(f"Processed pair {pair}, total points after merging: {len(load_data)}, ({original_rows} original total rows)")
            # append rows (block)
            big_table = pd.concat([big_table, load_data],
                                  axis=0,
                                  ignore_index=True)


Processed pair GISP2-NG1, total points after merging: 290, (290 original total rows)
Processed pair GISP2-NG2, total points after merging: 619, (725 original total rows)
Processed pair GISP2-NEEM, total points after merging: 194, (194 original total rows)
Processed pair GISP2-GRIP, total points after merging: 825, (918 original total rows)
Processed pair NG1-NG2, total points after merging: 311, (311 original total rows)
Processed pair NG1-NEEM, total points after merging: 578, (673 original total rows)
Processed pair NG1-GRIP, total points after merging: 530, (587 original total rows)
Processed pair NG2-NEEM, total points after merging: 824, (886 original total rows)
Processed pair NG2-GRIP, total points after merging: 907, (1003 original total rows)
Processed pair NEEM-GRIP, total points after merging: 396, (396 original total rows)


In [ ]:
from numpy._core.numeric import indices


core_groups = defaultdict(list)
matching_groups = defaultdict(list)

link_columns = [col for col in big_table.columns if "reference" not in col]

for col in link_columns:
    suffix = col.split("_")[-1]
    core_groups[suffix].append(col) #group cols by suffix
    
    match = col.split("_")[0]
    core1 = match.split("-")[0]
    core2 = match.split("-")[1]

    if core1 == suffix:
        matching_core = core2
    elif core2 == suffix:
        matching_core = core1

    matching_groups[suffix].append(f"{match}_{matching_core}")

update_check = 0  # total number of filled-in values across all passes
refresh = 1  # triggers loop until no new updates are found
while refresh > 0:  # keep looping as long as new values were added
    refresh = 0  # reset per loop
    for core, assoc_cols in core_groups.items():  # columns associated with each core
        matching_cols = matching_groups[core]  # corresponding matching columns

        for col, match_col in zip(assoc_cols, matching_cols):  # pair actual vs matching column
            base_col = col.split("_")[0] # get base link name for reference column
            ref_col = f"{base_col}_reference"  # reference column name
            for col_check in assoc_cols:  # compare against all other columns of same core
                if col == col_check:  # skip self-comparison
                    continue

                col_updates = {}  # values to update in primary column
                match_updates = {}  # values to update in matching column
                ref_updates = {}  # values to update in reference column

                for index, value in big_table[col].items():  # loop over each row
                    diff = (big_table[col_check] - value).abs()  # compute absolute diff
                    matching_indices = diff[diff <= error_margin].index  # rows that agree within margin

                    for match_idx in matching_indices:  # for each compatible row
                        col_updates[match_idx] = big_table[col].at[index]  # schedule update for col
                        match_updates[match_idx] = big_table[match_col].at[index]  # schedule update for match_col
                        ref_updates[match_idx] = big_table[ref_col].at[index]  # schedule update for reference column
            
                for match_idx, new_val in col_updates.items():  # apply col updates
                    if pd.isna(big_table.at[match_idx, col]):  # only fill empty cells
                        big_table.at[match_idx, col] = new_val  # write new value
                        update_check+=1  # count total updates
                        refresh+=1  # signal another full loop is needed
                for match_idx, new_val in match_updates.items():  # apply match_col updates
                    if pd.isna(big_table.at[match_idx, match_col]):  # only fill empty cells
                        big_table.at[match_idx, match_col] = new_val  # write new value
                for match_idx, new_val in ref_updates.items():  # apply reference column updates
                    if pd.isna(big_table.at[match_idx, ref_col]):  # only fill empty cells
                        big_table.at[match_idx, ref_col] = new_val  # write new value
            
    print(f'total updates made: {update_check} (+{refresh})')  # show total and new updates this pass

#deal with with duplicates
non_ref_cols = [c for c in big_table.columns if "reference" not in c]
big_table[non_ref_cols] = big_table[non_ref_cols].round(8)
duplicates_mask = big_table.duplicated(subset=non_ref_cols, keep='first')
num_dupe = duplicates_mask.sum()
big_table_cleaned = big_table.drop_duplicates(subset=non_ref_cols, keep='first').reset_index(drop=True)
print(f'Reduced table by {num_dupe/len(big_table)*100:.2f}% due to duplicates')

#reorganize based on ages
core_chron = {}
for core, columns in core_groups.items():
    file_path = f'/Users/quinnmackay/Documents/GitHub/BICC/Antarctic Chronology Accuracy Project/{project}/Chronologies/{core}.txt'
    df = pd.read_csv(file_path, sep="\t", comment="#", names=['depth', 'age']).sort_values(by=['depth']).reset_index(drop=True)
    core_chron[core] = df

for index, row in big_table_cleaned.iterrows():
    for core, columns in core_groups.items(): # For each core and its associated list of column names
        if core != base_core_age:
            continue
# for index, row in big_table_cleaned.iterrows():
#     row_age=[]
#     for core, columns in core_groups.items(): # For each core and its associated list of column names
#         values = [] # Collect the values for this core on this row
#         values = [row[col] for col in columns if not pd.isna(row[col])] # Remove NaN values so they don't interfere with comparison, get values for this core on this row
#         if len(values) >= 1:
#             avg_depth = np.mean(values)
#             chron_df = core_chron[core]
#             age_core_row = np.interp(avg_depth, chron_df['depth'], chron_df['age'])
#             row_age.append(age_core_row)
#     avg_row_age = np.mean(row_age)
#     big_table_cleaned.at[index, 'estimated_age'] = avg_row_age

# big_table_cleaned = big_table_cleaned.sort_values(by=['estimated_age']).reset_index(drop=True)
# big_table_cleaned = big_table_cleaned.drop(columns=['estimated_age'])


# Do evaluation for errors (inter-row errors)
within_row_errors = []
within_row_errors_core = []
within_row_big_errors = []
within_row_big_errors_core = []
for index, row in big_table_cleaned.iterrows(): # Iterate over every row in the table
    for core, columns in core_groups.items(): # For each core and its associated list of column names
        values = []
        for col in columns: # Collect the values for this core on this row
            values.append(row[col])
        values = [v for v in values if not pd.isna(v)] # Remove NaN values so they don't interfere with comparison
        if len(values) >= 2:
            diff = abs(max(values) - min(values))
            if diff >= error_margin:
                within_row_errors.append(index)
                within_row_errors_core.append(core)
                #print(f"Row {index} core {core} values: {values} diff={diff}")
            if diff >= big_error_margin and index not in within_row_big_errors:
                within_row_big_errors.append(index)
print(f'Identified rows with small within-row errors, {len(within_row_errors) - len(within_row_big_errors)} total')
print(f'Identified rows with big within-row errors, {len(within_row_big_errors)} total')

# # Eval between row errors
# for index, row in big_table_cleaned.iterrows(): # Iterate over every row in the table
#     if index not in within_row_errors:
#         continue
#     core_grab = [within_row_errors_core[within_row_errors.index(index)]]
#     between_indices = [i for i, v in enumerate(within_row_errors) if v == index]
#     cores = [within_row_errors_core[i] for i in between_indices]
#     for core, columns in core_groups.items(): # For each core and its associated list of column names
#         if core not in cores:
#             continue
        

#move cols around
reference_cols = [c for c in big_table_cleaned.columns if "reference" in c]
other_cols = [c for c in big_table_cleaned.columns if "reference" not in c]
big_table_cleaned = big_table_cleaned[other_cols + reference_cols]

rename_map = {}
for suffix, cols in core_groups.items():
    for col in cols:
        rename_map[col] = suffix  # rename to suffix only
big_table_cleaned.rename(columns=rename_map, inplace=True)
print('Renamed all columns to their suffix')

index_v = True
min_cols_per = {}

min_cols_export = 0
excel_path = f'{output_dir}/{project}_full.xlsx'
big_table_cleaned = big_table_cleaned[big_table_cleaned.notna().sum(axis=1) >= min_cols_export]
big_table_cleaned.to_excel(excel_path, index=index_v)
min_cols_per[excel_path] = min_cols_export
print(f'Exported cleaned table to excel at {excel_path}')

min_pairs_export = 2
excel_path = f'{output_dir}/{project}_{min_pairs_export}plus.xlsx'
min_cols_export = ((min_pairs_export-1) * 3) + 1 # y = x - 1 (so its minimum cols above the max of 1 less pair), times 3 cols per pair, plus 1 to be above
filtered_big_table = big_table_cleaned[big_table_cleaned.notna().sum(axis=1) >= min_cols_export]
filtered_big_table.to_excel(excel_path, index=index_v)
min_cols_per[excel_path] = min_pairs_export
print(f'Exported 3+ filtered table to excel at {excel_path}')

total updates made: 19461 (+19461)
total updates made: 20204 (+743)
total updates made: 20238 (+34)
total updates made: 20238 (+0)
Reduced table by 68.89% due to duplicates
Identified rows with small within-row errors, 92 total
Identified rows with big within-row errors, 9 total
Renamed all columns to their suffix
Exported cleaned table to excel at /Users/quinnmackay/Desktop/table_out/Greenland_full.xlsx
Exported 3+ filtered table to excel at /Users/quinnmackay/Desktop/table_out/Greenland_2plus.xlsx


In [5]:
excel_paths = list(min_cols_per.keys())

#thick side
thick_side = Side(border_style="thick", color="000000") 
thick_left_border = Border(left=thick_side)
medium_side = Side(border_style="medium", color="000000")
bottom_medium_border = Border(bottom=medium_side)
medium_left_border = Border(left=medium_side)
medium_right_border = Border(right=medium_side)
medium_border = Border(top=medium_side, left=medium_side, right=medium_side, bottom=medium_side)
light_side = Side(border_style="thin", color="000000")
light_left_border = Border(left=light_side)

for excel_path in excel_paths:
    wb = load_workbook(excel_path)
    ws = wb.active
    print(f"Loaded workbook {excel_path} for styling")

    #create second page for legend/stats
    legend_sheet = wb.create_sheet(title="Legend, Stats, and References")
    legend_sheet["A1"] = "Legend and Stats"
    legend_sheet["A1"].font = Font(size=14, bold=True)
    legend_sheet["A1"].alignment = Alignment(horizontal="center")
    legend_sheet.merge_cells('A1:D1')  # Merge first row for title
    for col_idx in range(1, 5):
        legend_sheet.cell(row=1, column=col_idx).border = medium_border
    legend_sheet.freeze_panes = "A2"

    #### Legend sheet reference move

    # Find all columns with 'reference' in header (case-insensitive)
    reference_cols = [i+1 for i, cell in enumerate(ws[1]) if cell.value and "reference" in str(cell.value).lower()]

    if reference_cols:
        # Copy index column header and data (assumes index col is 1)
        legend_sheet.cell(row=1, column=7).value = "Index"
        legend_sheet.cell(row=1, column=7).font = Font(bold=True)
        legend_sheet.cell(row=1, column=7).alignment = Alignment(horizontal="center", vertical="center")
        legend_sheet.cell(row=1, column=7).border = medium_border

        for row_idx in range(2, ws.max_row + 1): #index copy
            legend_sheet.cell(row=row_idx, column=7).value = ws.cell(row=row_idx, column=1).value
            legend_sheet.cell(row=row_idx, column=7).alignment = Alignment(horizontal="center", vertical="center")
            legend_sheet.cell(row=row_idx, column=7).border = medium_border
            legend_sheet.cell(row=row_idx, column=7).fill = PatternFill(start_color="FFFFBA", end_color="FFFFBA", fill_type='solid')  # light yellow fill
        # Copy each reference column header and data starting from column 7 (F=6, so G=7 etc)
        #rename reference headers
        for offset, col_idx in enumerate(reference_cols, start=8):
            original_header = ws.cell(row=1, column=col_idx).value
            # Split by '_' and keep the first part
            new_header = original_header.split('_')[0] if original_header else ""
            legend_sheet.cell(row=1, column=offset).value = new_header
            legend_sheet.cell(row=1, column=offset).alignment = Alignment(horizontal="center", vertical="center")
            legend_sheet.cell(row=1, column=offset).font = Font(bold=True)
            legend_sheet.cell(row=1, column=offset).border = bottom_medium_border
            
            offset_letter = get_column_letter(offset)
            legend_sheet.column_dimensions[offset_letter].width = 25
            for row_idx in range(2, ws.max_row + 1): #value copy
                legend_sheet.cell(row=row_idx, column=offset).value = ws.cell(row=row_idx, column=col_idx).value
                legend_sheet.cell(row=row_idx, column=offset).border = light_left_border
        # Remove reference columns from main sheet, delete from right to left
        for col_idx in sorted(reference_cols, reverse=True):
            ws.delete_cols(col_idx)

    ####### Legend Dicts

    # Add legend entries
    legend = {
        "ffd966": "Rows flagged with values differing by > 0.1 but all less than < 0.25",
        "e06666": "Rows flagged with values maximum differing by > 0.25",
    }

    legend_row = 3
    legend_sheet[f"A{legend_row}"] = "Legend"
    legend_sheet[f"A{legend_row}"].font = Font(bold=True)

    legend_row +=1
    for key, desc in legend.items():
        cell = legend_sheet[f"A{legend_row}"]
        cell.fill = PatternFill(start_color=key, end_color=key, fill_type='solid')  # Set fill style
        legend_sheet[f"B{legend_row}"] = desc
        legend_row += 1

    #count how many error on this sheet
    num_minor = 0
    num_major = 0
    for row_idx in range(2, ws.max_row + 1):
            cell = ws.cell(row=row_idx, column=1)
            if cell.value in within_row_errors:
                num_minor += 1
            if cell.value in within_row_big_errors:
                num_major += 1

    # Add some stats
    stats = {
        "Total Rows": ws.max_row - 1,  # assuming ws is your main sheet, -1 for header
        "Total Rows with Errors": f"{num_minor} ({num_minor / (ws.max_row - 1) * 100:.2f}%)",
        "Total Minor Errors (excl. Major)": f"{num_minor - num_major} ({(num_minor - num_major) / (ws.max_row - 1) * 100:.2f}%)",
        "Total Major Errors": f"{num_major} ({num_major / (ws.max_row - 1) * 100:.2f}%)",
        "Minimum Columns per Row": f"{min_cols_per[excel_path]}",
    }

    stats_row = legend_row + len(legend)
    legend_sheet[f"A{stats_row}"] = "Statistics"
    legend_sheet[f"A{stats_row}"].font = Font(bold=True)

    stats_row +=1
    for stat, val in stats.items():
        legend_sheet[f"A{stats_row}"] = stat
        legend_sheet[f"B{stats_row}"] = val
        stats_row += 1

    #adjust readability
    legend_sheet.column_dimensions['A'].width = 25  # wider column A in legend
    legend_sheet.column_dimensions['B'].width = 25  # wider column A in legend

    ### Styling main sheet ----------

    # Load headers
    headers = [cell.value for cell in ws[1]]

   # error coloring and error column
    max_columns = ws.max_column
    if index_v:
        headers_to_color = headers[1:]
        start_col = 2   # Excel column index: 1 = index col, 2 = real col 1
        ws["A1"].value = "Index"
        ws["A1"].font = Font(bold=True)
        ws["A1"].alignment = Alignment(horizontal="center", vertical="center")

        for row_idx in range(2, ws.max_row + 1):
            cell = ws.cell(row=row_idx, column=1)
            cell.font = Font(bold=False)
            cell.alignment = Alignment(horizontal="center", vertical="center")
            if cell.value in within_row_errors:
                cell.fill = PatternFill(start_color="ffd966", end_color="ffd966", fill_type='solid')

                error_cell = ws.cell(row=row_idx, column=max_columns + 1)
                indices = [i for i, v in enumerate(within_row_errors) if v == cell.value]
                cores = [within_row_errors_core[i] for i in indices]
                error_cell.value = ", ".join(cores)

            if cell.value in within_row_big_errors:
                cell.fill = PatternFill(start_color="e06666", end_color="e06666", fill_type='solid')
        error_column = get_column_letter(max_columns + 1)
        ws.column_dimensions[error_column].width = 25
        print(f'Added error corrections for {os.path.basename(excel_path)}')

    else:
        headers_to_color = headers
        start_col = 1

    # Define distinct light colors
    colors = [
        "FFB3BA", "FFDFBA", "FFFFBA", "BAFFC9", "BAE1FF",
        "D7BAFF", "FFC3F7", "BAFFD9", "FFE0BA", "D0BAFF"
    ]

    colors_note = [
    "#FFB3BA", "#FFDFBA", "#FFFFBA", "#BAFFC9", "#BAE1FF",
    "#D7BAFF", "#FFC3F7", "#BAFFD9", "#FFE0BA", "#D0BAFF"]

    # Assign colors to unique header names
    color_map = {}
    for col_name in headers_to_color:
        if col_name not in color_map:
            color_map[col_name] = colors[len(color_map) % len(colors)]

    # Apply fill color to each column
    col_idx = start_col
    for col_name in headers_to_color:

        fill = PatternFill(
            start_color=color_map[col_name],
            end_color=color_map[col_name],
            fill_type='solid'
        )

        # Color header
        ws.cell(row=1, column=col_idx).fill = fill

        # Color all data rows
        for row_idx in range(2, ws.max_row + 1):
            ws.cell(row=row_idx, column=col_idx).fill = fill

        col_idx += 1

    # Loop through columns starting at column 3 (Excel index), applying thick border every 2 columns 
    for col_idx in range(2, ws.max_column + 2, 2): # 3, 5, 7, 9 ... 
        for row_idx in range(1, ws.max_row + 1):
            ws.cell(row=row_idx, column=col_idx).border = thick_left_border

    # Medium thick on bottom of row 1
    medium_border_int = 1 if index_v else 0
    for col_idx in range(1, ws.max_column + medium_border_int):
        ws.cell(row=1, column=col_idx).border = bottom_medium_border

    #freeze top row
    ws.freeze_panes = "A2"

    #add error column header
    if index_v:
        error_col_cell = ws.cell(row=1, column=max_columns + 1)
        error_col_cell.value = "Error Cores"
        error_col_cell.font = Font(bold=True)
        error_col_cell.alignment = Alignment(horizontal="center", vertical="center")

    #rename sheet
    ws.title = "Ice Core Depth Comparison"

    # Save workbook
    wb.save(excel_path)
    print(f"Styled and saved workbook at {excel_path}")


Loaded workbook /Users/quinnmackay/Desktop/table_out/Greenland_full.xlsx for styling
Added error corrections for Greenland_full.xlsx
Styled and saved workbook at /Users/quinnmackay/Desktop/table_out/Greenland_full.xlsx
Loaded workbook /Users/quinnmackay/Desktop/table_out/Greenland_2plus.xlsx for styling
Added error corrections for Greenland_2plus.xlsx
Styled and saved workbook at /Users/quinnmackay/Desktop/table_out/Greenland_2plus.xlsx
